In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('/Users/alexy161/Downloads/Headlines.txt', sep='@')

In [ ]:
import nltk
import re
import string

alphanumeric = lambda x: re.sub(r"""\w*\d\w*""", ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())
stemmer = nltk.stem.LancasterStemmer()
stop_words = nltk.corpus.stopwords.words('english')


df.article = df.article.map(alphanumeric).map(punc_lower)
df.article = df.article.apply(nltk.word_tokenize)
df.article = df.article.apply(lambda x: [word for word in x if word not in stop_words])
df.article = df.article.apply(lambda x: [(stemmer.stem(word)) for word in x])

df

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
df['article_words'] = df['article'].apply(lambda x: ' '.join(map(str, x)))
x = df.article_words
cv1 = CountVectorizer(stop_words='english')
df_cv = cv1.fit_transform(x)

In [ ]:
pd.DataFrame(df_cv.toarray(), columns=cv1.get_feature_names())

In [ ]:
from textblob import TextBlob
df['sentiment'] = ''
df['sentiment'] = df.article_words.apply(lambda x: pd.Series(TextBlob(x).sentiment))
df['positive'] = np.where(df['sentiment'] > 0, 1, 0)
df['negative'] = np.where(df['sentiment'] < 0, 1, 0)
df

In [ ]:
dfa = df.groupby('stock').mean().reset_index()
dfa.sort_values(by='sentiment', ascending=True)

In [ ]:
st = pd.read_csv('/Users/alexy161/Downloads/Stock Revenues.csv')
st = st[['stock', 'q2', 'q1']]
st['q2_change'] = (st['q2'] - st['q1']) / st['q1']
st

In [ ]:
df1 = pd.merge(st, dfa, on='stock')
df1

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
X = df1[['q2_change', 'sentiment', 'positive', 'negative']]
stand = StandardScaler()
X = stand.fit_transform(X)
reducer = PCA(n_components=2)
X = reducer.fit_transform(X)

In [ ]:
plt.scatter(df1.q2_change, df1.sentiment)
plt.title('Change in Sales and Headline Sentiment')
plt.xlabel('Q2 Sales Change')
plt.ylabel('Headline Sentiment');

In [ ]:
num_clusters = 5
km = KMeans(n_clusters=num_clusters,init='k-means++',n_init=10)
km.fit(X)
c_label = km.predict(X)

In [ ]:
plt.scatter(df1.q2_change, df1.sentiment, c=c_label)
plt.title('Clusters for Change in Sales and Headline Sentiment')
plt.xlabel('Q2 Sales Change')
plt.ylabel('Headline Sentiment');

In [ ]:
inertia = []
list_num_clusters = list(range(1,10))
for num_clusters in list_num_clusters:
    km = KMeans(n_clusters=num_clusters)
    km.fit(X)
    inertia.append(km.inertia_)
    
plt.plot(list_num_clusters,inertia)
plt.scatter(list_num_clusters,inertia)
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia');

In [ ]:
X, c_label

In [ ]:
plt.scatter(df1.q2_change, df1.positive, c=c_label)
plt.title('Clusters for Change in Sales and Positive Headline Frequency')
plt.xlabel('Q2 Sales Change')
plt.ylabel('Positive Headline Frequency');

In [ ]:
plt.scatter(df1.q2_change, df1.negative, c=c_label)
plt.xlabel('Q2 Change')
plt.ylabel('Negative');